# Setup

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np

from utilities import my_show, my_gshow, my_read, my_read_g, my_read_cg, size_me

img_dir = '../common/'
# keep np.array display under control
np.set_printoptions(precision=4)

%matplotlib inline

# Calculus in Pixel-space

## Image Derivatives

In [ ]:
box = np.zeros((15, 15), dtype=np.int8)
box[4:11, 4:11] = 1.0
my_gshow(plt.gca(), box, interpolation=None)

In [ ]:
from utilities import my_gshow

box = np.zeros((15, 15), dtype=np.int8)
box[4:11, 4:11] = 1.0
my_gshow(plt.gca(), box, interpolation=None)

In [ ]:
# here's a line across the middle .... 
line = box[5:6, :] # 5:6 to keep 2D
line_p = np.diff(line) # line "prime" aka derivative

print(line)
print(line_p)
print(line_p.shape)

In [ ]:
# here's a line across the middle .... 
line = box[5:6, :] # 5:6 to keep 2D


print(line)
print(line_p, line_p.shape)

In [ ]:
# note, derivative has values [-1, 0, 1] ... 
# these get mapped to [0, 128, 255] 
# (aka, black, gray, white) inside of imshow

fig,axes = plt.subplots(2, 1, figsize=(6,1), sharex=True)
my_gshow(axes[0], line, interpolation=None)
my_gshow(axes[1], line_p)

## Integral Image

It's not too hard to construct integral images out of NumPy primatives.  It's also good practice with NumPy!

In [ ]:
arr = np.arange(1,4*4+1).reshape(4,4)
print(arr)
integral = arr.cumsum(axis=1).cumsum(axis=0)
integral

In [ ]:
pt1 = (1,1)
pt2 = (2,2)
print(6 + 7 + 10 + 11,
      integral[2,2] - integral[0,2] - integral[2,0] + integral[0,0])

In [ ]:
def make_integral_img(img):
    return img.cumsum(axis=1).cumsum(axis=0)

def area(integral_image, pt1, pt2):
    # assume pt1 is the outer point and fix if necessary
    outer_pt, inner_pt = pt1, pt2
    if sum(pt1) < sum(pt2):
        outer_pt, inner_pt = inner_pt, outer_pt

    # the squares
    outer_sq  = integral_image[outer_pt]
    inner_sq  = integral_image[inner_pt[0]-1, inner_pt[1]-1]
    
    # the rectangles
    tall_rect = integral_image[outer_pt[0], inner_pt[1]-1]
    wide_rect = integral_image[inner_pt[0]-1, outer_pt[1]]
    
    # inclusion-exclusion
    return outer_sq - tall_rect - wide_rect + inner_sq

img = np.arange(1,4*4+1).reshape(4,4)
integral_img = make_integral_img(img)

pt1, pt2 = (2,2), (1,1)

area(integral_img, pt1, pt2)

# An Aside:  Kernel (aka Neighborhood or Local Region) Methods

In [ ]:
# probabilty of a 2 .... given by 
# (1) taking all combinations of events, 
# (2) computing sums, 
# (3) return counts of 2 events divided by total number of events
# also, we know this is 1/6 * 1/6
d1, d2 = np.meshgrid(*[np.arange(1,7)]*2)
sums = d1 + d2
event_table = dict(zip(*np.unique(d1+d2, return_counts=True)))
print("{:.4f} {:.4f}".format(event_table[2] / sums.size, 1/6 * 1/6))

In [ ]:
event_table[2]

In [ ]:
# probability of a 2:  
# P_twodice(Total) = sum_part P_onedie(part) * P_onedie(Total-part)
# P_twodice(2) = sum_part P_onedie(part) * P_onedie(2-part)
# [note, the only valid part here is part = 1 ... all others are "too big"]
die = np.full(6, 1/6.0)
print(die)

In [ ]:
# can think:  invert second, align, multiple and add
np.convolve(die, die, mode='full') # all probabilities in event space (pads outside with effective zeros)

In [ ]:
# in general, the pdf of a sum of events is the convolution of the component events!

In [ ]:
# another way to think about convolutions is as "sliding windows"
# in the example above, to get, say a total of 7
# we can fix a point 7 and slide the two arrays past (one in reverse order) ... 
# and when they add up to seven, we take that sum-product

# say we flip the second array and start running them past each other
# we start with a 6,6 ... then we 6,5; 5,6 .... and so on
# eventually, they are lined up such that we get 1,6; ... 6,1 ...
# *all* of our ways of getting 7 ... we can take that dot-product and we have our 
# probability

In [ ]:
# another example is taking a moving average
# since the second array is "symmetric" about its middle
# this greatly simplifies our mental arithmetic  ... 
# we just slide it past, computing dot-products (weighted sums) as we go
# our "total" is simply the center point we are at ... 
#       aka, we align the weight-window centered at the point we are trying to fill
values = np.arange(10.0)
kernel = np.full(3, 1/3)
print(values, kernel, 
      np.convolve(values, kernel, mode='valid'), # you can try full/same here and see the edge effects
      sep='\n')

print("*" * 20)

values = np.array([3,6,9,3,12,15])
print(values, kernel, 
      np.convolve(values, kernel, mode='valid'),
      sep='\n')

In [ ]:
# that's all great, but we are working with images.  we can do the same thing in two-D
from scipy.signal import convolve2d as ss_convolve2d
box = np.zeros((15, 15), dtype=np.int8)
box[4:11, 4:11] = 1.0

size = 3 # adjust me!
kernel = np.full((size,size), 1/size**2)
out = ss_convolve2d(box, kernel, mode='same') 
# with same or full, we have to pad ... 
#      can use wrapping (around the image)
#      symmetric (mirroring back from the edge)
#      fill value

print(out.shape)
my_gshow(plt.gca(), out, interpolation=None)

In [ ]:
# and here's what's happening in the top-left corner of the white box:
import seaborn as sns

box = np.zeros((5,5), np.uint8)
box[3:, 3:] = 1.0
kernel = np.full((3,3), 1/3**2)
out = ss_convolve2d(box, kernel, mode='same', boundary='symm')

fig, axes = plt.subplots(1,3,figsize=(12,4))

kwargs = dict(annot=True, fmt=".3f", cbar=False)
sns.heatmap(box,    ax=axes[0], **kwargs)
sns.heatmap(kernel, ax=axes[1], **kwargs)
sns.heatmap(out,    ax=axes[2], **kwargs)

[ax.axis('off') for ax in axes]
fig.tight_layout();

# experiment with different padding to see what happens to the bottom-right square

# Proc:  Morphology

In [ ]:
img = np.zeros((500,500), dtype=np.uint8)
circle = cv2.circle(img,(250,250), 100, 255, -1)

blur = np.random.randint(0,256,size=img.shape).astype(np.uint8)
blurred = np.where(np.random.uniform(size=img.shape) > .25, img, blur)

my_show(plt.gca(), blurred, cmap='gray')

In [ ]:
# "erode" away the foreground (foreground == white)
# pixel is on if entire kernel-neighborhood is on
# so, inside is good, outside is off
#     borders of foreground:  will be reduced
#     more will become background
# enhances background; removes noise in background ... add noise in foreground

# "dilate" adds to the foreground (white)
# pixel is on if ANY kernel-neighborhood is on
# inside - good; outside - off; border - expanded
# enhances foreground; removes noise in foreground ... adds noise in background

fig, axes = plt.subplots(2,3, figsize=(12,8))
fig.tight_layout()
kernel = np.ones((7, 7), dtype=np.uint8)

for idx, base in enumerate([circle, blurred]):
    eroded = cv2.erode(base, kernel, iterations = 3)
    dilated = cv2.dilate(base, kernel, iterations = 3)

    my_gshow(axes[idx,0], base,    title='original image')
    my_gshow(axes[idx,1], eroded,  title='eroded')
    my_gshow(axes[idx,2], dilated, title='dilated')

In [ ]:
# opening: dilate(erode(img))  ... aka, erode it, then dilate it
# remove outside noise (false foreground); remove local peaks
# count objects
# opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)

# closing:  erode(dilate(img))
# remove inside noise (false background)
# used as a step in connected-components analysis
# closing = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)

# ^^^^ iterations of these are erode(erode(dilate(dilate())))
#      i.e. e^i(d^i(img))

# gradient = dilation - erosion ... finds bounary
# gradient = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel)

# these two isolate brigher/dimmer (tophat/blackhat) than their surroundings
# tophat:  image - opening
# tophat = cv2.morphologyEx(img, cv2.MORPH_TOPHAT, kernel)
# blackhat:  closing - image
# blackhat = cv2.morphologyEx(img, cv2.MORPH_BLACKHAT, kernel)

# Blurring and Smoothing

In [ ]:
img = cv2.imread(img_dir+'data/opencv-logo.png')

kernel_size = (5,5)
kernel = np.full(kernel_size, 1/np.prod(kernel_size))
dst = cv2.filter2D(img,-1,kernel)

print(img.shape, dst.shape)

fig,axes = plt.subplots(1,2,figsize=(8,6))
my_show(axes[0], img)
my_show(axes[1], dst)

In [ ]:
blur = cv2.blur(img, kernel_size) # average of kernel_size neighborhood
boxf  = cv2.boxFilter(img, -1, kernel_size, normalize=False) #???

fig,axes = plt.subplots(1,3,figsize=(12,6))
my_show(axes[0], img)
my_show(axes[1], blur)
my_show(axes[2], boxf)

In [ ]:
gauss  = cv2.GaussianBlur(img, kernel_size, 5)


dots = np.random.randint(0,256,size=img.shape).astype(np.uint8)
dotted = np.where(np.random.uniform(size=img.shape) > .3, img, dots)
median = cv2.medianBlur(dotted, kernel_size[0])  # 5 --> 5x5 neighborhood

fig,axes = plt.subplots(2,2,figsize=(12,12))
my_show(axes[0,0], img)
my_show(axes[0,1], dotted)
my_show(axes[1,0], gauss)
my_show(axes[1,1], median)

In [ ]:
# need a better image example for bilateral filtering
# bilateral refers to smoothing both intensities AND colors
# "edge preserving".  produces a "water color painting effect" when repeated
saved = cv2.bilateralFilter(dotted,9,100, 50)
my_show(plt.gca(), saved)

# Proc: Image Pyramids

In [ ]:
# unsure of object sizes in an image - 
# work with images of different resolutions and find objects in each
# "stack" of images is an image pyramid

# gaussian:  lower resolution  <---- higher resolution by removing rows/cols then gaussian

In [ ]:
messi = my_read(img_dir+'data/messi.jpg')

fig, ax = plt.subplots(1,1,figsize=size_me(messi))
my_show(ax, messi)

# pyrDown == "reduce" == downSample(gaussian(img)) == downSample(gaussian conv. img)
gaussian_pyr_1 = cv2.pyrDown(messi)
fig, ax = plt.subplots(1,1,figsize=size_me(gaussian_pyr_1))
my_show(ax, gaussian_pyr_1)

gaussian_pyr_2 = cv2.pyrDown(gaussian_pyr_1)
fig, ax = plt.subplots(1,1,figsize=size_me(gaussian_pyr_2))
my_show(ax, gaussian_pyr_2)

In [ ]:
cv2

In [ ]:
restored = cv2.pyrUp(cv2.pyrUp(gaussian_pyr_2))  # back to full size ... lost information
fig, ax = plt.subplots(1,1,figsize=size_me(restored))
my_show(ax, restored)

In [ ]:
# laplacian L_lvl = G_lvl - expanded(G_{lvl + 1})
# https://www.cs.utah.edu/~arul/report/node12.html
laplacian_messi = messi - cv2.pyrUp(cv2.pyrDown(messi))
fig, ax = plt.subplots(1,1,figsize=size_me(laplacian_messi))
my_show(ax, laplacian_messi)

In [ ]:
messi.shape

In [ ]:
g_0 = messi[:256, :512] # powers of two to make halving/doubling happy; could also pad out to next power of two
g_1 = cv2.pyrDown(g_0) # REDUCE(messi)
g_2 = cv2.pyrDown(g_1)

l_0 = g_0 - cv2.pyrUp(g_1)   # by simple algebra:  img = g0 = l_0 + UP(g1)
l_1 = g_1 - cv2.pyrUp(g_2)
base = g_2

print(g_1.shape, l_1.shape)

# more algebra:
# g_0 = l_0 + UP(g_1) = l_0 + UP(l_1 + UP(g2)) = l_0 + UP(l_1 + UP(base))

# typically stored the "laplacian pyramid" which is l_0 .... l_n-1 and the "base" which is g_n
# then conceptually (not strict addition):  base + l_n-1 + l_0 --> original
# sort of a x_0 + diffs representation

restored_1 = l_0 + cv2.pyrUp(g_1)
restored_2 = l_0 + cv2.pyrUp(l_1 + cv2.pyrUp(base))
fig, ax = plt.subplots(1,3,figsize=(12,4))
my_show(ax[0], g_0)
my_show(ax[1], restored_1)
my_show(ax[2], restored_2)



# Pyramids for Blending

In [ ]:
# FIXME:  this could be an exercise answer
#         i.e., do the "manual" process with apple/orange
#         that we showed with Messi (but improved by generate_pyramids)
apple = my_read(img_dir+'data/apple.png')
orange = my_read(img_dir+'data/orange.png')

def generate_pyramids(img, lvls):
    gp = [img.copy()]
    lp = []
    for i in range(lvls-1):
        curr_gp = gp[-1]
        next_gp = cv2.pyrDown(curr_gp)
        next_lp = curr_gp - cv2.pyrUp(next_gp, dstsize=curr_gp.shape[:2])
        lp.append(next_lp)
        gp.append(next_gp)

    lp.append(gp[-1])
    return gp,lp

NUM_LVLS = 6

gp_a, lp_a = generate_pyramids(apple, NUM_LVLS)
gp_o, lp_o = generate_pyramids(orange, NUM_LVLS)

print("\n".join(str([gp.shape, lp.shape]) for gp,lp in zip(gp_a, lp_a)))

# manually recreate from gp_2 --> ... and from gp_3 --> ...
restored_apple_1 = lp_a[0] + cv2.pyrUp(lp_a[1] + cv2.pyrUp(gp_a[2]))
restored_apple_2 = lp_a[0] + cv2.pyrUp(lp_a[1] + cv2.pyrUp(lp_a[2] + cv2.pyrUp(gp_a[3])))
fig, ax = plt.subplots(1,2,figsize=(12,4))
my_show(ax[0], restored_apple_1)
my_show(ax[1], restored_apple_2)

In [ ]:
def add_upped(a, b):
    return cv2.pyrUp(a, dstsize=b.shape[:2]) + b
    
def generate_pyramids(img, lvls):
    gp = [img.copy()]
    lp = []
    for i in range(lvls-1):
        curr = gp[-1]
        next_ = cv2.pyrDown(curr)
        next_lp = curr - cv2.pyrUp(next_, dstsize=curr.shape[:2])
        lp.append(next_lp)
        gp.append(next_)

    lp.append(gp[-1])
    return gp,lp

import functools as ft
def reconstruct(lp_mask, lp_left, lp_right):
    blended_lp = []
    for lp_m, lp_l, lp_r in zip(lp_mask, lp_left, lp_right):
        wgt = lp_m.astype(np.float64) # FIXME.  this appears to have no effect
        new_lvl = (wgt * lp_l) + ((1-wgt) * lp_r)
        blended_lp.append(new_lvl.astype(np.uint8))
    return ft.reduce(add_upped, reversed(blended_lp))    

In [ ]:
fig, axes = plt.subplots(2,2,figsize=(8,8))
fig.tight_layout()

apple = my_read(img_dir+'data/apple.png')
orange = my_read(img_dir+'data/orange.png')
mask = np.zeros((240,240,3), dtype=np.uint8)
mask[:, 120:, :] = 1
# mask[:, :120] = 1  # there are some artifacts using this mask

my_show(axes[0,0], apple)
my_show(axes[0,1], orange)

NUM_LVLS = 7
gp_a, lp_a = generate_pyramids(apple, NUM_LVLS)
gp_o, lp_o = generate_pyramids(orange, NUM_LVLS)
mask_gp, _ = generate_pyramids(mask, NUM_LVLS)

    
final = reconstruct(mask_gp, lp_a, lp_o)
my_show(axes[1,0], final, title='pyramid merge')

cols = apple.shape[1]
raw_merge = np.hstack([orange[:,:cols//2],
                       apple[:,cols//2:]])
my_show(axes[1,1], raw_merge, title='raw merge')

# Image Gradients via Filters

In [ ]:
box = np.zeros((500,500), dtype=np.uint8)
box[150:350, 150:350] = 1.0
my_gshow(plt.gca(), box)

We can apply Sobel and Laplacian filters directly with opencv.  For the Sobel filters, teh arguments are which direction to compute.  So, `sobel_x` is assigned a Sobel filter that responds to vertical gradient changes (walking across rows of the underlying matrix).  `sobel_y` responds to horizontal changes (walking up/down columns of the matrix).

In [ ]:
# this code is written simply, but if we have to update anything, ugh
laplacian = cv2.Laplacian(box, cv2.CV_64F, ksize=5)
sobel_x   = cv2.Sobel(box, cv2.CV_64F, 1, 0, ksize=5)
sobel_y   = cv2.Sobel(box, cv2.CV_64F, 0, 1, ksize=5)
sobel_xy  = cv2.Sobel(box, cv2.CV_64F, 1, 1, ksize=5)

fig, axes = plt.subplots(1,5,figsize=(12,3))
for ax, smoother in zip(axes.flat,
                        [box,laplacian, sobel_x, sobel_y, sobel_xy]):
    my_gshow(ax, smoother)

In [ ]:
fig, axes = plt.subplots(1,5,figsize=(12,3))
axes = axes.flat
common_args = {'ddepth':cv2.CV_64F, 'ksize':5}

# original and laplacian
my_gshow(next(axes), box)
my_gshow(next(axes), cv2.Laplacian(box, **common_args))

# sobel filters; note (1,1) has very faint contents in corners
for ax, (dx, dy) in zip(axes, [(1,0), (0,1), (1,1)]):
    full_args = dict(common_args, dx=dx, dy=dy)
    sob = cv2.Sobel(box, **full_args)
    my_gshow(ax, sob)

print("Top left corner values:\n", sob[147:153, 147:153])

If we needed to compare these edge detectors many times, we might want to (1) write a helper function to encapsulate the process and (2) deal with issues like keeping both edges as "positive" values. Remember how we had +1 and -1 when we computed np.diff above - we can address that by taking the absolute value of the edges we find.  The net result is both the "white-to-black" and "black-to-white" have a response of 1.  Also, we can threshold the results to force black and white images for increased contrast.

In [ ]:
def show_filters(orig, axes, both_edges=False, enhance=False, **common_args):
    if both_edges: # to keep edges, cv needs to keep sign info
        common_args['ddepth'] = cv2.CV_64F
    my_gshow(next(axes), orig)

    sobel_args = [(1,0), (0,1), (1,1)]
    sobels = [cv2.Sobel(orig, **dict(common_args, dx=dx, dy=dy)) for  dx,dy in sobel_args]
    filters = [cv2.Laplacian(orig, **common_args)] + sobels
    
    if both_edges:
        filters = [np.absolute(f).astype(np.uint8) for f in filters]
    if enhance:
        filters = [cv2.threshold(f, 1, 255, cv2.THRESH_BINARY)[1] for f in filters]
    
    for f in filters:
        my_gshow(next(axes), f)
    return filters

In [ ]:
box = np.zeros((500,500), dtype=np.uint8)
box[150:350, 150:350] = 1.0
fig, axes = plt.subplots(1,5,figsize=(12,3))

# note, 8U implies unsigned ... clipped at 0
results = show_filters(box, axes.flat, both_edges=True, 
                       enhance=True, ddepth=cv2.CV_8U, ksize=5)

fig,axes = plt.subplots(1,5,figsize=(12,3))
my_gshow(axes[0], np.uint8([[255]]),vmin=0) # bleh:  black out first square
for f, ax in zip(results, axes[1:]):
    my_gshow(ax, f[125:175, 125:175], interpolation=None)
    ax.set_title('Nonzero: {}'.format(np.count_nonzero(f)))

In [ ]:
# so that was painful, right?  well, now we can resuse it.  the next cell is easy!
height, width = box.shape
M = cv2.getRotationMatrix2D((width/2,height/2), 45, 1)
diamond = cv2.warpAffine(box, M, (width, height))

fig, axes = plt.subplots(1,5,figsize=(12,3))

show_filters(diamond, axes.flat, both_edges=True, enhance=True, ddepth=cv2.CV_8U, ksize=5);

# Distance Transform

Once we have edges, we may need to find and group together pixels as a-hypothetical-object (although we don't really have any concept beyond "groups of pixels"). One step in that process is to find the distances from a pixel to a boundary.  Two interesting distance metrics to use here are:
  * `cv2.DIST_L2`.  The L2 norm:  Euclidean distance, aka pythagorean theorem), 
  * `cv2.DIST_L1`.  Chessboard distance for a rook that can only move one square at a time:   allows moves on row and col. 
  * `cv2.DIST_C`.   Chessboard distance for a king: allows moves on row, col, and diagonal.

In [ ]:
# image -> threshold -> edges -> invert (non-edge is white)
sobel_xy  = np.absolute(cv2.Sobel(diamond, cv2.CV_64F, 1, 1, ksize=5)).astype(np.uint8)
sobel_xy = cv2.threshold(sobel_xy, 1, 255, cv2.THRESH_BINARY)[1]
np.unique(sobel_xy, return_counts=True)
sobel_xy = cv2.subtract(255,sobel_xy) # invert
my_gshow(plt.gca(), sobel_xy)

# distances from a point to a white point
dist = cv2.distanceTransform(sobel_xy, cv2.DIST_L2, 0)
my_gshow(plt.gca(), dist, interpolation=None)

In [ ]:
# label is grouping to nearest edge (both sides of edge grouped together)
dist, labels = cv2.distanceTransformWithLabels(sobel_xy, cv2.DIST_L2, 5)
print(labels.shape,
      labels[:5,:5],
      labels[250:255, 250:255], 
      (labels.min(), labels.max()),sep="\n*******\n")
my_gshow(plt.gca(), labels, interpolation=None)

# Exercises

##### Gradients on the Diagonal

Create an image of several concentric rings and take the derivative of the image across its diagonal.  Hint:  check out `np.diag`!

##### Morphology

In [ ]:
# FIXME EXERCISE
# create a small staircase and apply morphology to it
# try different ops and kernels
# we used a square kernel.  could do rectangle "easily".  for other kernels:
# Rectangular  Elliptical  Cross kernels
# cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))
# cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
# cv2.getStructuringElement(cv2.MORPH_CROSS,(5,5))

##### Morphology and Spy Games

In [ ]:
# bigger project includes morphology
# from LOCV pg 142
# image with/without object
#      morph_open(thresh(abs(w - wo))
# ---> noisy mask for object
#      floodfill, keep largest only (see LOCV: pg 124)
# ---> clean mask for object
# ---> insert object into another photo

##### Visualizing Pyramids

Consider five different gaussian pyramids:

  * (1) one on the rgb image, 
  * (1) on the grayscale of the RGB image, and 
  * (3) one on each of the three channels in isolation

Create a series of subplots for these over three pyramid levels.  This will give a grid of 20 total images if you include the originals.  Do the same for a Laplacian pyramid.  Use three levels of "real" Laplacians, with the base level being a Laplacian, not the Gaussian you might store for reconstruction.

In [ ]:
fig, axes = plt.subplots(4,5,figsize=(15,12))
fig.tight_layout()

apple, apple_g = my_read_cg(img_dir+'data/apple.png')

def one_channel(img, c):
    ' fixme:  seem hackish, but it sure works! '
    out = np.zeros_like(img)
    out[:,:,c] = img[:,:,c]
    return out

# Student section here 



In [ ]:
fig, axes = plt.subplots(4,5,figsize=(15,12))
fig.tight_layout()

apple, apple_g = my_read_cg(img_dir+'data/apple.png')

# Student section here 



# Filters

Create a series of rectangles with narrower and narrower widths down to a single pixel line.  Apply a Sobel filter to these.  If Sobel filters find "edges", how many edges does the single pixel rectangle (that is, a "line") have?  Try using kernel sizes of 5 and 3.  Look at the numeric Sobel output values along a horizontal slice of the image.

In [ ]:

widths = [1,2,4,8]
height = 20

# Student section here 



Additional practice ideas:

  * Research and implement a roberts edge detector
      * apply it to binary image
      * add noise - apply it.  
      * https://dsp.stackexchange.com/questions/898/roberts-edge-detector-how-to-use
  
  * Predict what a distance transform would do to a checkerboard
    * Try it out!